In [2]:
'''
Author: Ngo Van Uc
Date: 23/08/2024
Gmail: ngovanuc.1508@gmail.com
'''

'\nAuthor: Ngo Van Uc\nDate: 23/08/2024\nGmail: ngovanuc.1508@gmail.com\n'

# 00. Import

In [3]:
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from ast import literal_eval

import numpy as np

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 01. Load dataset

In [5]:
train_path = '/content/drive/MyDrive/RESEARCHING/ASPECT_BASE_SENTIMENT_ANALYSIS/data/Train.csv'
test_path = '/content/drive/MyDrive/RESEARCHING/ASPECT_BASE_SENTIMENT_ANALYSIS/data/Test.csv'
dev_path = '/content/drive/MyDrive/RESEARCHING/ASPECT_BASE_SENTIMENT_ANALYSIS/data/Dev.csv'

train_data = pd.read_csv(train_path)
train_data.head()

,index,comment,n_star,date_time,label
0,0,Mới mua máy này Tại thegioididong thốt nốt cảm...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,1,Pin kém còn lại miễn chê mua 8/3/2019 tình trạ...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,2,Sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,3,17/08/2020,{FEATURES#Negative};
3,3,"Mọi người cập nhật phần mềm lại , nó sẽ bớt tố...",3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,4,"Mới mua Sài được 1 tháng thấy pin rất trâu, Sà...",5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


# 02. Preprocessing

In [6]:
class_to_id = {"SCREEN": 0, "CAMERA": 1, "FEATURES": 2, "BATTERY": 3, "PERFORMANCE": 4, "STORAGE": 5, "DESIGN": 6, "PRICE": 7, "GENERAL": 8, "SER&ACC": 9}
label_to_id = {"Positive": 0, "Negative": 1, "Neutral": 2, "None": 3}

In [7]:
def get_label_dict(sentences_label):
    label_dict = {}
    labels = sentences_label.split(';')[:-1]
    if len(labels) == 1 and labels[-1][1:-1] == "OTHERS":
        # đầu ra chỉ có OTHERS
        return None
    else:
        if labels[-1][1:-1] == "OTHERS":
            # đầu ra có OTHERS và các nhãn khác nữa
            for l in labels[:-1]:
                class_name, sentiment = l[1:-1].split('#')
                label_dict[class_name] = sentiment
            return label_dict
        else:
            for l in labels:
                class_name, sentiment = l[1:-1].split('#')
                label_dict[class_name] = sentiment
            return label_dict

In [8]:
sentences = train_data['comment'].tolist()
label_comment = train_data['label'].tolist()
others_label = [[0, 0, 0, 1]]*10

train_set = []

for sentence, sentence_label in zip(sentences, label_comment):
    label_dict = get_label_dict(sentence_label)
    labels_ = []
    if label_dict:
        for aspect in list(class_to_id.keys()):
            train_label = [0] * 4
            if aspect in label_dict.keys():
                label_id = label_to_id[label_dict[aspect]]
                train_label[label_id] =  1
                labels_.append(train_label)
            else:
                labels_.append([0, 0, 0, 1])
        # train_labels.append(labels_)
        train_set.append(
            {'sentence': sentence, 'labels': labels_}
        )
    else:
        train_set.append(
            {'sentence': sentence, 'labels': others_label}
        )
print(train_set[0]['sentence'])
print(train_set[0]['labels'])

Mới mua máy này Tại thegioididong thốt nốt cảm thấy ok bin trâu chụp ảnh đẹp loa nghe to bắt wf khỏe sóng ổn định, giá thành vừa với túi tiền, nhân viên tư vấn nhiệt tình
[[0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0]]


In [9]:
for i, data in enumerate(train_set):
    sentence, label = data.items()
    input = np.array(label[-1])
    clear_output()
    if input.shape == (10, 4):
        print("pass")
    else:
        print(i)

pass


# 03. Build model

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer


class AspectSentimentDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sentence = self.data[index]['sentence']
        labels = self.data[index]['labels']

        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'sentence_text': sentence,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.float)
        }

In [11]:
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-multilingual-cased')
dataset = AspectSentimentDataset(train_set, tokenizer, max_len=128)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [12]:
dataset.__len__()

7786

In [13]:
import torch.nn as nn
from transformers import BertModel


class AspectSentimentModel(nn.Module):
    def __init__(self, n_classes, n_labels_per_class):
        super(AspectSentimentModel, self).__init__()
        self.bert = BertModel.from_pretrained('google-bert/bert-base-multilingual-cased')
        self.lstm = nn.LSTM(768, 256, batch_first=True, bidirectional=True)
        self.classifier = nn.Linear(256*2, n_classes * n_labels_per_class)

    def forward(self, input_ids, attention_mask):
        bert_outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        lstm_outputs, _ = self.lstm(bert_outputs.last_hidden_state)
        lstm_outputs = lstm_outputs[:, -1, :]
        output = self.classifier(lstm_outputs)
        return output.view(-1, 10, 4)

model = AspectSentimentModel(n_classes=10, n_labels_per_class=4)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

# 04. Training model

In [14]:
import torch.optim as optim
import numpy as np


def train_epoch(model, data_loader, loss_fn, optimizer, device):
    model = model.train()
    losses = []
    correct_predictions = 0

    for data in data_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        loss = loss_fn(outputs, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return np.mean(losses)


EPOCHS = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
training = True


optimizer = optim.Adam(model.parameters(), lr=2e-5)
loss_fn = nn.BCEWithLogitsLoss()

if training == True:
    for epoch in range(EPOCHS):
        train_loss = train_epoch(model, dataloader, loss_fn, optimizer, device)
        print(f'Epoch {epoch + 1}/{EPOCHS}, Train loss: {train_loss}')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Epoch 1/10, Train loss: 0.35485674584670723
Epoch 2/10, Train loss: 0.2511550738520201
Epoch 3/10, Train loss: 0.21357102817578483
Epoch 4/10, Train loss: 0.18683833829308927
Epoch 5/10, Train loss: 0.1645154110490664
Epoch 6/10, Train loss: 0.14728355060429055
Epoch 7/10, Train loss: 0.13158436953348301
Epoch 8/10, Train loss: 0.11936823307733516
Epoch 9/10, Train loss: 0.10791112686788522
Epoch 10/10, Train loss: 0.09833912419067271


In [ ]:
path_save_model = '/content/drive/MyDrive/RESEARCHING/ASPECT_BASE_SENTIMENT_ANALYSIS'
torch.save(model.state_dict(), path_save_model+'/model_v2.pth')

In [ ]:
# model = AspectSentimentModel(n_classes=10, n_labels_per_class=4)
# model.load_state_dict(torch.load(path_save_model+'/model.pth'))
# model.eval()
# clear_output()

In [15]:
def predict_sentiment(model, sentence, tokenizer, max_len, device):
    model = model.eval()

    encoding = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    return outputs


def interpret_predictions(predictions):
    aspect_classes = ['SCREEN', 'CAMERA', 'FEATURES', 'BATTERY', 'PERFORMANCE', 'STORAGE', 'DESIGN', 'PRICE', 'GENERAL', 'SER&ACC']
    sentiment_labels = ['Positive', 'Negative', 'Neutral', 'None']

    results = {}

    for i, aspect in enumerate(aspect_classes):
        aspect_sentiments = predictions[0, i].cpu().numpy()
        sentiment_index = aspect_sentiments.argmax()
        results[aspect] = sentiment_labels[sentiment_index]

    return results

# 05. Thực hiện predict trên bộ dữ liệu kiểm thử - test

In [16]:
test_data = pd.read_csv(test_path)
test_data.head()

,index,comment,n_star,date_time,label
0,0,"Điện thoải ổn. Facelock cực nhanh, vân tay ôk ...",5,5/2/2020,{SCREEN#Positive};{FEATURES#Positive};{PERFORM...
1,1,"Mình mới mua vivo91c. Tải ứng dụng ,games nh...",5,14/05/2019,{FEATURES#Negative};{PERFORMANCE#Positive};{SE...
2,2,Xấu đẹp gì ko biết nhưng rất ưng TGdđ phục vụ ...,5,26/03/2020,{DESIGN#Neutral};{SER&ACC#Positive};
3,3,Màn hình hơi lác khi chơi game. Game nặng thì ...,4,4/6/2019,{PERFORMANCE#Negative};{DESIGN#Negative};{OTHE...
4,4,"Nói chung máy đẹp với màn amoled, ổn trong tầm...",4,12/5/2020,{SCREEN#Positive};{BATTERY#Negative};{DESIGN#P...


In [17]:
max_len = 256

idx = 20
sentence_test = test_data.loc[idx, 'comment']
true_label = test_data.loc[idx, 'label']
print(sentence_test)
print(true_label)

predictions = predict_sentiment(model, sentence_test, tokenizer, max_len, device)
results = interpret_predictions(predictions)

print('Aspect Sentiment Prediction:')
for aspect, sentiment in results.items():
    print(f'{aspect}: {sentiment}')

Mọi thứ ok.mình thấy pin trâu chứ có sao đâu.mua đc 2 tuần rồi chỉ có cảm biến vân tay ko nhạy như mong đợi thôi.
{FEATURES#Negative};{BATTERY#Positive};{GENERAL#Positive};
Aspect Sentiment Prediction:
SCREEN: None
CAMERA: None
FEATURES: Negative
BATTERY: Positive
PERFORMANCE: None
STORAGE: None
DESIGN: None
PRICE: None
GENERAL: Positive
SER&ACC: None


In [18]:
predictions_on_test = []

for i in range(len(test_data)):
    print(i)
    sentence_on_test = test_data['comment'][i]
    prediction_on_test = predict_sentiment(model, sentence_on_test, tokenizer, max_len, device)
    results = interpret_predictions(prediction_on_test)
    predictions_on_test.append(results)
    clear_output()

print('All done!')


All done!


In [19]:
i = 5
print(predictions_on_test[i])
print(test_data['label'][i])

{'SCREEN': 'None', 'CAMERA': 'None', 'FEATURES': 'Negative', 'BATTERY': 'None', 'PERFORMANCE': 'None', 'STORAGE': 'None', 'DESIGN': 'None', 'PRICE': 'None', 'GENERAL': 'Positive', 'SER&ACC': 'None'}
{SCREEN#Negative};{FEATURES#Negative};{GENERAL#Positive};


In [20]:
true_labels = []

for i in range(len(test_data)):
    outcome = {}
    label = test_data['label'][i]
    label = label.replace('{', '')
    label = label.replace('}', '')
    labels = label.split(';')
    for l in labels:
        try:
            text = l.split('#')
            outcome[text[0]] = text[1]
        except:
            pass
    true_labels.append(outcome)

In [21]:
print(len(true_labels))

idx = 6
print(f'Comment:\n{test_data["comment"][idx]}')
print(f'True label:\n{true_labels[idx]}')
print(f'Predicted: \n{predictions_on_test[idx]}')

2224
Comment:
Thật tuyệt máy qua mượt túi thích  như thế giới Di Động có chỗ giá khác túi không hiểu đánh giá 5sao cho thế giới Di Động phương Phú Châu 5sao nhân viên OK
True label:
{}
Predicted: 
{'SCREEN': 'None', 'CAMERA': 'None', 'FEATURES': 'None', 'BATTERY': 'None', 'PERFORMANCE': 'Positive', 'STORAGE': 'None', 'DESIGN': 'None', 'PRICE': 'None', 'GENERAL': 'Positive', 'SER&ACC': 'Positive'}


# 06. Chuẩn hóa nhãn đầu ra để thực hiện đánh giá hiệu suất mô hình

In [22]:
def full_label(true_label, prediction):
    '''true_label: dict
     prediction: dict '''
    for key, value in prediction.items():
        if key not in true_label:
            true_label[key] = 'None'
    return true_label

In [23]:
# mục tiêu là đưa danh sách các nhãn thực tế về dạng đánh giá 10 class như trong dự đoán
true_full_labels = []
for idx in range(len(test_data)):
    true_label = true_labels[idx]
    prediction = predictions_on_test[idx]
    true_full_labels.append(full_label(true_label, prediction))

In [24]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [25]:
# data thỉnh thoảng bị sai -..-

print(predictions_on_test[6])
print(true_labels[6])
print(test_data.loc[6, 'comment'])
for key, value in true_labels[6].items():
    print('true: ',key, value)
    print('predicted: ', key, predictions_on_test[6][key])

{'SCREEN': 'None', 'CAMERA': 'None', 'FEATURES': 'None', 'BATTERY': 'None', 'PERFORMANCE': 'Positive', 'STORAGE': 'None', 'DESIGN': 'None', 'PRICE': 'None', 'GENERAL': 'Positive', 'SER&ACC': 'Positive'}
{'SCREEN': 'None', 'CAMERA': 'None', 'FEATURES': 'None', 'BATTERY': 'None', 'PERFORMANCE': 'None', 'STORAGE': 'None', 'DESIGN': 'None', 'PRICE': 'None', 'GENERAL': 'None', 'SER&ACC': 'None'}
Thật tuyệt máy qua mượt túi thích  như thế giới Di Động có chỗ giá khác túi không hiểu đánh giá 5sao cho thế giới Di Động phương Phú Châu 5sao nhân viên OK
true:  SCREEN None
predicted:  SCREEN None
true:  CAMERA None
predicted:  CAMERA None
true:  FEATURES None
predicted:  FEATURES None
true:  BATTERY None
predicted:  BATTERY None
true:  PERFORMANCE None
predicted:  PERFORMANCE Positive
true:  STORAGE None
predicted:  STORAGE None
true:  DESIGN None
predicted:  DESIGN None
true:  PRICE None
predicted:  PRICE None
true:  GENERAL None
predicted:  GENERAL Positive
true:  SER&ACC None
predicted:  SER&A

# 07. Đánh giá hiệu suất tổng thể mô hình

In [26]:
# đoạn code này đánh giá hiệu suất của toàn bộ mô hình trên toàn bộ các đặc trưng
true_ = []
predicted_ = []

for idx in range(len(true_labels)):
    for key, value in predictions_on_test[idx].items():
        if true_labels[idx][key] == 'None' and value == 'None':
            # đầu ra 0, dự đoán 0 => TP
            true_.append(0)
            predicted_.append(0)
        elif true_labels[idx][key] != 'None' and true_labels[idx][key] == value:
            # đầu ra 1, dự đoán 1 => TN
            true_.append(1)
            predicted_.append(1)
        elif true_labels[idx][key] == 'None' and true_labels[idx][key] != value:
            # đầu ra 0, dự đoán 1 => FP
            true_.append(0)
            predicted_.append(1)
        elif true_labels[idx][key] != 'None' and true_labels[idx][key] == value:
            # đầu ra 1, dự đoán 0 => FN
            true_.append(1)
            predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9663459213771094
Precision = 0.8812083973374296
Recall = 1.0
f1 = 0.9368535655960806


# 08. Đánh giá hiệu suất nhận diện khía cạnh

### Accuracy trên toàn bộ khía cạnh

In [27]:
true_ = []
predicted_ = []
accuracy = []

for idx in range(len(predictions_on_test)):
    for key, value in predictions_on_test[idx].items():
        if true_labels[idx][key] == 'None' and value == 'None':
            accuracy.append(1)
        elif true_labels[idx] != 'None' and value != 'None':
            accuracy.append(1)
        elif true_labels[idx][key] == 'None' and value != 'None':
            accuracy.append(0)
        elif true_labels[idx][key] != 'None' and value == 'None':
            accuracy.append(0)


print(f'Accuracy = {sum(accuracy)/len(accuracy)}')

Accuracy = 0.9645233812949641


### Precision, Recall, F1-score trên toàn bộ khía cạnh

In [28]:
true_ = []
predicted_ = []

for idx in range(len(predictions_on_test)):
    for key, value in predictions_on_test[idx].items():
        if true_labels[idx] == 'None' and value == 'None':
            true_.append(0)
            predicted_.append(0)
        elif true_labels[idx][key] != 'None' and value != 'None':
            true_.append(1)
            predicted_.append(1)
        elif true_labels[idx][key] == 'None' and value != 'None':
            true_.append(0)
            predicted_.append(1)
        elif true_labels[idx][key] != 'None' and value == 'None':
            true_.append(1)
            predicted_.append(0)


print(f'Precision = {precision_score(true_, predicted_)}')
print(f'Recall = {recall_score(true_, predicted_)}')
print(f'F1-score = {f1_score(true_, predicted_)}')

Precision = 0.8950067883542012
Recall = 0.8826242189824457
F1-score = 0.8887723766010037


### đánh giá hiệu suất khía cạnh trên class SCREEN

In [29]:
true_ = []
predicted_ = []

for idx in range(len(true_labels)):
    key = 'SCREEN'
    if true_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9550359712230215
Precision = 0.8393574297188755
Recall = 0.7769516728624535
f1 = 0.8069498069498069


### đánh giá hiệu suất khía cạnh trên class FEATURES

In [30]:
true_ = []
predicted_ = []

for idx in range(len(true_labels)):
    key = 'FEATURES'
    if true_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9163669064748201
Precision = 0.8650904033379694
Recall = 0.8748241912798875
f1 = 0.8699300699300699


### đánh giá hiệu suất khía cạnh trên class PERFORMANCE

In [32]:
true_ = []
predicted_ = []

for idx in range(len(true_labels)):
    key = 'PERFORMANCE'
    if true_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.8920863309352518
Precision = 0.9153297682709447
Recall = 0.8762798634812287
f1 = 0.8953792502179598


### đánh giá hiệu suất khía cạnh trên class GENERAL

In [61]:
true_ = []
predicted_ = []

for idx in range(len(true_labels)):
    key = 'GENERAL'
    if true_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.845773381294964
Precision = 0.8450797872340425
Recall = 0.9203475742215785
f1 = 0.8811091854419411


### đánh giá hiệu suất khía cạnh trên class BATTERY

In [62]:
true_ = []
predicted_ = []

for idx in range(len(true_labels)):
    key = 'BATTERY'
    if true_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9694244604316546
Precision = 0.9530651340996169
Recall = 0.9812623274161736
f1 = 0.966958211856171


### đánh giá hiệu suất khía cạnh trên class STORAGE

In [63]:
true_ = []
predicted_ = []

for idx in range(len(true_labels)):
    key = 'STORAGE'
    if true_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9878597122302158
Precision = 0.0
Recall = 0.0
f1 = 0.0


### đánh giá hiệu suất khía cạnh trên class DESIGN

In [65]:
true_ = []
predicted_ = []

for idx in range(len(true_labels)):
    key = 'DESIGN'
    if true_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9348021582733813
Precision = 0.9174917491749175
Recall = 0.6984924623115578
f1 = 0.7931526390870185


### đánh giá hiệu suất khía cạnh trên class PRICE

In [66]:
true_ = []
predicted_ = []

for idx in range(len(true_labels)):
    key = 'PRICE'
    if true_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9482913669064749
Precision = 0.8900343642611683
Recall = 0.9103690685413005
f1 = 0.900086880973067


### đánh giá hiệu suất khía cạnh trên class SER&ACC

In [67]:
true_ = []
predicted_ = []

for idx in range(len(true_labels)):
    key = 'SER&ACC'
    if true_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9109712230215827
Precision = 0.8957915831663327
Recall = 0.7537942664418212
f1 = 0.8186813186813187


### đánh giá hiệu suất khía cạnh trên class CAMERA

In [68]:
true_ = []
predicted_ = []

for idx in range(len(true_labels)):
    key = 'CAMERA'
    if true_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9716726618705036
Precision = 0.9324546952224053
Recall = 0.9625850340136054
f1 = 0.9472803347280334


# 09. Đánh giá hiệu suất nhận diện tình cảm (đa nhãn)

### Accuracy trên toàn bộ data kiểm thử
acc = corrects/(all predictions)

In [69]:
# đánh giá độ chính xác accuracy (chỉ dựa trên nhãn true_labels để đánh giá)
true_ = []
predicted_ = []
accuracy = []

for idx in range(len(true_labels)):
    for key, value in true_labels[idx].items():
        if value == 'None':
            continue
        elif predictions_on_test[idx][key] != value:
            accuracy.append(0)
        elif predictions_on_test[idx][key] == value:
            accuracy.append(1)

print(f'Accuracy = {sum(accuracy)/len(accuracy)}')

Accuracy = 0.7680749776852127


### Precisio, Recall, F1-score trên toàn bộ data kiểm thử

In [70]:
from sklearn.metrics import classification_report


y_true = []
y_pred = []

for idx in range(len(true_labels)):
    for key, value in true_labels[idx].items():
        y_true.append(key + "_" + value)
        y_pred.append(key + "_" + predictions_on_test[idx][key])


print(f'Classification report:')
print(classification_report(y_true, y_pred, digits=4))

Classification report:
                      precision    recall  f1-score   support

    BATTERY_Negative     0.7441    0.9402    0.8307       368
     BATTERY_Neutral     0.0000    0.0000    0.0000        92
        BATTERY_None     0.9839    0.9595    0.9715      1210
    BATTERY_Positive     0.8722    0.9116    0.8914       554
     CAMERA_Negative     0.6816    0.8012    0.7366       171
      CAMERA_Neutral     0.4000    0.0282    0.0526        71
         CAMERA_None     0.9864    0.9749    0.9806      1636
     CAMERA_Positive     0.8080    0.9364    0.8675       346
     DESIGN_Negative     0.6364    0.0729    0.1308        96
      DESIGN_Neutral     0.0000    0.0000    0.0000        28
         DESIGN_None     0.9375    0.9863    0.9613      1826
     DESIGN_Positive     0.7842    0.8358    0.8092       274
   FEATURES_Negative     0.7466    0.8410    0.7910       459
    FEATURES_Neutral     0.0000    0.0000    0.0000        52
       FEATURES_None     0.9409    0.9359    0

In [71]:
precision_micro = precision_score(y_true, y_pred, average='micro')
recall_micro = recall_score(y_true, y_pred, average='micro')
f1_micro = f1_score(y_true, y_pred, average='micro')

precision_macro = precision_score(y_true, y_pred, average='macro')
recall_macro = recall_score(y_true, y_pred, average='macro')
f1_macro = f1_score(y_true, y_pred, average='macro')

precision_weighted = precision_score(y_true, y_pred, average='weighted')
recall_weighted = recall_score(y_true, y_pred, average='weighted')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

print("\nOverall Precision, Recall, F1-Score:")
print(f"Micro Precision: {precision_micro:.4f}")
print(f"Micro Recall: {recall_micro:.4f}")
print(f"Micro F1-Score: {f1_micro:.4f}")
print(f"Macro Precision: {precision_macro:.4f}")
print(f"Macro Recall: {recall_macro:.4f}")
print(f"Macro F1-Score: {f1_macro:.4f}")
print(f"Weighted Precision: {precision_weighted:.4f}")
print(f"Weighted Recall: {recall_weighted:.4f}")
print(f"Weighted F1-Score: {f1_weighted:.4f}")


Overall Precision, Recall, F1-Score:
Micro Precision: 0.8986
Micro Recall: 0.8986
Micro F1-Score: 0.8986
Macro Precision: 0.6091
Macro Recall: 0.5910
Macro F1-Score: 0.5822
Weighted Precision: 0.8832
Weighted Recall: 0.8986
Weighted F1-Score: 0.8868


### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh SCREEN

In [72]:
y_true = []
y_pred = []

key = 'SCREEN'
for idx in range(len(true_labels)):
    y_true.append(true_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.72      0.58      0.64       116
     Neutral       0.00      0.00      0.00        17
        None       0.97      0.98      0.97      1955
    Positive       0.74      0.85      0.79       136

    accuracy                           0.94      2224
   macro avg       0.61      0.60      0.60      2224
weighted avg       0.93      0.94      0.94      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh FEATURES

In [73]:
y_true = []
y_pred = []

key = 'FEATURES'
for idx in range(len(true_labels)):
    y_true.append(true_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.75      0.84      0.79       459
     Neutral       0.00      0.00      0.00        52
        None       0.94      0.94      0.94      1513
    Positive       0.71      0.71      0.71       200

    accuracy                           0.87      2224
   macro avg       0.60      0.62      0.61      2224
weighted avg       0.86      0.87      0.87      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh PERFORMANCE

In [48]:
y_true = []
y_pred = []

key = 'PERFORMANCE'
for idx in range(len(true_labels)):
    y_true.append(true_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.75      0.80      0.78       454
     Neutral       0.50      0.01      0.02       116
        None       0.87      0.91      0.89      1052
    Positive       0.82      0.88      0.85       602

    accuracy                           0.83      2224
   macro avg       0.74      0.65      0.63      2224
weighted avg       0.81      0.83      0.81      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh GENERAL

In [49]:
y_true = []
y_pred = []

key = 'GENERAL'
for idx in range(len(true_labels)):
    y_true.append(true_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.65      0.77      0.70       294
     Neutral       0.00      0.00      0.00        83
        None       0.85      0.72      0.78       843
    Positive       0.83      0.95      0.88      1004

    accuracy                           0.80      2224
   macro avg       0.58      0.61      0.59      2224
weighted avg       0.78      0.80      0.79      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh CAMERA

In [50]:
y_true = []
y_pred = []

key = 'CAMERA'
for idx in range(len(true_labels)):
    y_true.append(true_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.68      0.80      0.74       171
     Neutral       0.40      0.03      0.05        71
        None       0.99      0.97      0.98      1636
    Positive       0.81      0.94      0.87       346

    accuracy                           0.93      2224
   macro avg       0.72      0.69      0.66      2224
weighted avg       0.92      0.93      0.91      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh BATTERY

In [51]:
y_true = []
y_pred = []

key = 'BATTERY'
for idx in range(len(true_labels)):
    y_true.append(true_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.74      0.94      0.83       368
     Neutral       0.00      0.00      0.00        92
        None       0.98      0.96      0.97      1210
    Positive       0.87      0.91      0.89       554

    accuracy                           0.90      2224
   macro avg       0.65      0.70      0.67      2224
weighted avg       0.88      0.90      0.89      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh STORAGE

In [52]:
y_true = []
y_pred = []

key = 'STORAGE'
for idx in range(len(true_labels)):
    y_true.append(true_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00         6
     Neutral       0.00      0.00      0.00         3
        None       0.99      1.00      0.99      2197
    Positive       0.00      0.00      0.00        18

    accuracy                           0.99      2224
   macro avg       0.25      0.25      0.25      2224
weighted avg       0.98      0.99      0.98      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh DESIGN

In [53]:
y_true = []
y_pred = []

key = 'DESIGN'
for idx in range(len(true_labels)):
    y_true.append(true_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.64      0.07      0.13        96
     Neutral       0.00      0.00      0.00        28
        None       0.94      0.99      0.96      1826
    Positive       0.78      0.84      0.81       274

    accuracy                           0.92      2224
   macro avg       0.59      0.47      0.48      2224
weighted avg       0.89      0.92      0.89      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh PRICE

In [54]:
y_true = []
y_pred = []

key = 'PRICE'
for idx in range(len(true_labels)):
    y_true.append(true_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.55      0.53      0.54        79
     Neutral       0.77      0.77      0.77       328
        None       0.97      0.96      0.97      1655
    Positive       0.67      0.73      0.70       162

    accuracy                           0.90      2224
   macro avg       0.74      0.75      0.74      2224
weighted avg       0.90      0.90      0.90      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh SER&ACC

In [55]:
y_true = []
y_pred = []

key = 'SER&ACC'
for idx in range(len(true_labels)):
    y_true.append(true_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.71      0.43      0.54       167
     Neutral       0.00      0.00      0.00        27
        None       0.92      0.97      0.94      1631
    Positive       0.87      0.87      0.87       399

    accuracy                           0.90      2224
   macro avg       0.63      0.57      0.59      2224
weighted avg       0.88      0.90      0.89      2224

